### https://riptutorial.com/julia-lang/example/20457/creating-expressions

In [3]:
MyExpr2 = :(x = 2)
eval(MyExpr2)

2

In [4]:
MyExpr3 = Expr(:(=), :x, 2)
MyExpr3 == MyExpr2

true

When using this syntax, it is important to distinguish between using objects and symbols for objects. For instance, in the above example, the expression assigns the value of `2` to the symbol `:x`, a perfectly sensible operation. If we used `x` itself in an expression such as that, we would get the nonsensical result:<br>

**在使用 `Expr()` 这种语法时，区分使用对象和对象符号是很重要的**。例如，在上面的例子中，表达式将 `2` 赋值给符号 `:x` ，这是一个非常合理的操作。如果在这样的表达式中使用 `x` 本身，就会得到无意义的结果：

In [5]:
Expr(:(=), x, 5)

:(2 = 5)

Thus, the `Expr()` function does not perform the same automatic transformation into symbols as the `:()` syntax for creating expressions.
因此， `Expr()` 函数不会像 `:()` 语法那样自动转换为符号来创建表达式。

### https://docs.juliacn.com/latest/manual/metaprogramming/#表达式与求值

In [1]:
dump(Meta.parse("1+1"))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1


In [2]:
Expr(:call, :+, 1, 1) == Meta.parse("1+1")

true

In [17]:
Meta.show_sexpr(:(a+b*c+1))

(:call, :+, :a, (:call, :*, :b, :c), 1)

In [18]:
Meta.parse("a + b*c + 1") ==
:(a + b*c + 1)       ==
Expr(:call, :+, :a, Expr(:call, :*, :b, :c), 1)

true

In [20]:
a=1;b=2;c=3

@show :(a + b*c + 1) == quote a + b*c + 1 end # 后者有多了行信息

println("\n")
@show eval(:(a + b*c + 1)) == eval(quote a + b*c + 1 end)

$(Expr(:quote, :(a + b * c + 1))) == $(Expr(:quote, quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:3 =#
    a + b * c + 1
end)) = false


eval($(Expr(:quote, :(a + b * c + 1)))) == eval($(Expr(:quote, quote
    #= e:\Projects.jl\Training.jl\9. Expr\jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X21sZmlsZQ==.jl:6 =#
    a + b * c + 1
end))) = true


true

In [5]:
ex = Expr()

ArgumentError: ArgumentError: Expr: too few arguments (expected 1)

In [6]:
ex = Expr(:call)

:($(Expr(:call)))

In [7]:
ex.args=[:+]

1-element Vector{Symbol}:
 :+

In [8]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((1,))
    1: Symbol +


In [9]:
append!(ex.args, [1, 2])

3-element Vector{Any}:
  :+
 1
 2

In [11]:
eval(ex)

3

### https://blog.csdn.net/m0_37696990/article/details/81986431

In [1]:
f = :foo
Expr(f, :+, 1, 1) |> display

g = :(:)
Expr(g, :+, 1, 1) |> display

:($(Expr(:foo, :+, 1, 1)))

:($(Expr(:(:), :+, 1, 1)))

### **\julia\test\syntax.jl中搜索 parse,** 有很多Expr示例

In [57]:
Meta.parse(":(x.\$f[i])")

:($(Expr(:quote, :(x.:($(Expr(:$, :f)))[i]))))

In [50]:
dump(Meta.parse(":(x.\$f[i])"))

Expr
  head: Symbol quote
  args: Array{Any}((1,))
    1: Expr
      head: Symbol ref
      args: Array{Any}((2,))
        1: Expr
          head: Symbol .
          args: Array{Any}((2,))
            1: Symbol x
            2: QuoteNode
              value: Expr
                head: Symbol $
                args: Array{Any}((1,))
        2: Symbol i


In [45]:
Meta.parse(":(x.\$f[i])") == Expr(:quote,
     Expr(:ref, 
          Expr(Symbol("."),
               :x,
               QuoteNode(Expr(:$,:f))),
          :i 
         )
)

true

In [46]:
using TreeView

In [54]:
TreeView.walk_tree(Meta.parse(":(x.\$f[i])"))

LabelledTree({6, 5} directed simple Int64 graph, Any[:quote, :ref, :., :x, :($(QuoteNode(:($(Expr(:$, :f)))))), :i])

! Extra }, or forgotten $.
<recently read> \egroup 
        
l.50 }
    ;


In [12]:
Expr(:$, :var)

:($(Expr(:$, :var)))

In [22]:
QuoteNode(Expr(:$, :var))

:($(QuoteNode(:($(Expr(:$, :var))))))

In [31]:
dump(:($(:var)))

Symbol var


In [28]:
var = 1
@show Expr(:$, :var) == :($var)
@show eval(Expr(:$, :var)) == eval(:($var))

Expr(:$, :var) == $(Expr(:quote, :($(Expr(:$, :var))))) = false


ErrorException: syntax: "$" expression outside quote

In [21]:
@show eval(:($(Expr(:$, :var)))) == eval(:($var))

ErrorException: syntax: "$" expression outside quote

In [2]:
Expr(:$, :(1+2))

:($(Expr(:$, :(1 + 2))))

In [1]:
dump(Expr(:$, :(1+2)))
# eval(Meta.quot(Expr(:$, :(1+2))))

Expr
  head: Symbol $
  args: Array{Any}((1,))
    1: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 1
        3: Int64 2


In [3]:
eval(Expr(:$, :(1+2)))

ErrorException: syntax: "$" expression outside quote

In [4]:
eval(:($(Expr(:$, :(1 + 2)))))

ErrorException: syntax: "$" expression outside quote

In [7]:
q = Meta.quot(Expr(:$, :(1+2)))

:($(Expr(:quote, :($(Expr(:$, :(1 + 2)))))))

In [8]:
dump(q)

Expr
  head: Symbol quote
  args: Array{Any}((1,))
    1: Expr
      head: Symbol $
      args: Array{Any}((1,))
        1: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol +
            2: Int64 1
            3: Int64 2


In [10]:
q2=
:(
    $(
        Expr(:quote, 
             :(
                $(
                    Expr(:$, :(1 + 2))
                 )
              )
             )
    )
)

:($(Expr(:quote, :($(Expr(:$, :(1 + 2)))))))

In [11]:
eval(q2)

3

In [12]:
c = :(1+2)
Expr(:$, :(1+2)) |> display
:($c) |> display
eval(:($c))

:($(Expr(:$, :(1 + 2))))

:(1 + 2)

3

In [16]:
Expr(:$, :(1+2)) |> display
eval(Expr(:$, :(1+2)))

:($(Expr(:$, :(1 + 2))))

ErrorException: syntax: "$" expression outside quote

In [ ]:
eval(:(Expr(:$, :(1+2)))) |> display
eval(eval(:(Expr(:$, :(1+2)))))

:($(Expr(:$, :(1 + 2))))

ErrorException: syntax: "$" expression outside quote